### Comparison of RNN Approaches
(see https://distill.pub/2016/augmented-rnns/)

This creates a sample task, based on number sequences, and
different goals to compare the performance of vanilla RNN's,
LSTM's, NTM's, and attention based networks.

In [1]:
import torch

In [2]:
from torch import nn, einsum
from random import randint
from torch.nn import functional as f

In [3]:
class Numbers:
    ''' This is a controller 
    
    '''
    def __init__(self, max_number = 9,
                 increment_func = lambda stream: stream[-1]+1,
                 reset_func = None,
                 reset_value_func = lambda stream: 0,
                 goal_func = lambda stream: stream[0]):
        ''' Initialize the controller
        Args:
            increment_func: called on each number in the stream to decide
                the next number in the stream
            reset_func: evalutes the stream to determine if a reset
                condition has been met causing a restart on a new number
            reset_value_func: if a reset is triggered, this is called to
                determine the new value to restart the stream with
            goal_func: a function that returns the target output value
                for the stream (i.e. the goal output)
        '''
        self.max_number = max_number
        self.all_numbers = list(range(max_number+1))
        self.all_classes = self.all_numbers + [-1]
        
        self.increment_func = increment_func
        self.reset_func = reset_func
        self.reset_value_func = reset_value_func
        self.goal_func = goal_func
        
        
    def create_stream(self, length, seed_stream = [0]):
        ''' Create a stream (sequence) of numbers
        
        Args:
            length: length of the stream to generate
            seed_stream: seed stream to use in generating the first number
                
        Returns:
            A sequence (list) of numbers in the stream
        '''
        if self.reset_func == None:
            self.reset_func = lambda stream: stream[-1] == self.max_number
    
        start_num = self.reset_value_func(seed_stream)
        stream = [start_num]
        
        for i in range(max(0,length-1)):
            if self.reset_func(stream):
                new_num = self.reset_value_func(stream)
            else:
                new_num = self.increment_func(stream)
            stream.append(new_num)
            
        return stream
    
    
    def encode_stream(self, num_stream):
        ''' Converts number or number_stream into one-hot encoded tensor
        An additional category is added for OTHER (unrecognized numbers)
        
        Args:
            num_stream: A single number of sequence (list) of numbers
                that will be encoded
            
        Returns:
            Encoded tensor of shape (number_index, _, one_hot_encoded_number)
        '''
        num_stream = num_stream if type(num_stream) is list else [num_stream]
        output = torch.zeros(len(num_stream), 1, len(self.all_numbers)+1)
        
        for i, num in enumerate(num_stream):
            value = num if num in self.all_numbers else -1
            ind = self.all_classes.index(value)
            output[i,0,ind] = 1
            
        return output
    
    def get_stream_goal(self, stream):
        ''' Return the goal output for the providede stream 
        Args:
            stream: a list of numbers in the sequence
        
        Returns:
            Simple evaluation of the goal function (type depends
            on the goal function provided)
        '''
        return self.goal_func(stream)
    
    
    def decode_row(self, encoded_number):
        ''' Decode a one-hot encoded vector back to the number '''
        ind = encoded_number.argmax()
        value = self.all_numbers[ind] if ind in self.all_numbers else -1
        return value
    
        
    def get_dim(self):
        ''' Return dimension of each encoded tensor '''
        return len(self.all_classes)

In [4]:
def train_model(model, number_tool, criterion, optim, train_size = 100e3, stream_size = 200, print_interval = 1e3):
    ''' Runs a full training pass for a given model '''
    
    train_size = int(train_size)
    print_interval = int(print_interval)
    error_sum = 0
    model_device = next(model.parameters()).device

    for i in range(train_size):
        # Generate a new random sequence for training
        stream = number_tool.create_stream(stream_size)
        obs = number_tool.encode_stream(stream).to(model_device)

        # Reset the model gradients and hidden layer
        model.zero_grad()
        hidden = model.init_hidden()
        
        # Score the model
        output, hidden = model.forward(obs, hidden)

        # Determine the target output and calculate the loss
        goal = number_tool.get_stream_goal(stream)
        target = torch.LongTensor([goal]).to(model_device)
        loss = criterion(output.squeeze(0), target)
        
        # Run backprop with the errors and update model/trackers
        loss.backward()
        optim.step()
        error_sum += loss

        if i % print_interval == 0:
            print(f'[{i}] Error: {error_sum / print_interval}')
            error_sum = 0

## Model Definitions

In [5]:
class BasicRNN(nn.Module):
    ''' Basic RNN with a softmax output layer '''
    
    def __init__(self, input_size, hidden_size, output_size):
        ''' Init the RNN '''
        super(BasicRNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.input_to_output = nn.Linear(input_size + hidden_size, output_size)
        self.input_to_hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.softmax = nn.LogSoftmax(dim=2)
        
    def forward(self, input, hidden):
        ''' Run a forward path for a single number of sequence of numbers
        
        Returns:
            output and hidden layer after last sequence input
        '''
        for i in range(input.shape[0]):
            combined_input = torch.cat((input[i].unsqueeze(0), hidden), dim=2)
            hidden = self.input_to_hidden(combined_input)
            
        output = self.input_to_output(combined_input)
        output = self.softmax(output)
        
        return output, hidden
    
    def init_hidden(self):
        ''' Returns new hidden layers for the start of a new sequence '''
        model_device = next(self.parameters()).device
        return torch.zeros(1, 1, self.hidden_size).to(model_device)

In [93]:
class BasicLSTM(nn.Module):
    ''' Basic single-LSTM with output calculated using a single
        linear layer with a softmax activation '''
    
    def __init__(self, input_size, hidden_size, output_size):
        ''' Init the LSTM '''
        super(BasicLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.hidden_to_output = nn.Linear(hidden_size, output_size)
        self.hidden_size = hidden_size
        self.softmax = nn.LogSoftmax(dim=2)
        
    def forward(self, input, hidden):
        ''' Run a forward path for a single number of sequence of numbers
        
        Returns:
            output and hidden layer after last sequence input
        '''
        # Hidden is made up of (hidden, cell_state)
        history, hidden = self.lstm.forward(input, hidden)
        output = self.hidden_to_output(hidden[0])
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        ''' Returns new hidden layers for the start of a new sequence '''
        model_device = next(self.parameters()).device
        return (
            torch.randn(1, 1, self.hidden_size).to(model_device),
            torch.randn(1, 1, self.hidden_size).to(model_device)
        )

In [109]:
class Memory:
    ''' Implements an attention-based memory system as outlined in
        the Neural Turing Machine paper '''
    
    def __init__(self, banks, dim):
        ''' Initialize the memory store
        
        Args:
            banks: number of memory banks
            dim: dimension of each memory bank
        '''
        self.memory = torch.zeros(banks, dim)
        
        
    def read(self, weights):
        ''' Reads stored memory based on the attention
        
        Args:
            weights: attention for each cell (size: banks)
            
        Returns:
            weighted memory output (size: dim)
        '''
        attention = f.normalize(weights, p=1, dim=0)
        read = einsum('ij,i->j', (self.memory, attention))
        return read
    
    
    def write(self, weights, erase, add):
        ''' Updates attented memory with the erase and add
        
        Args:
            weights: attention paid to each cell (size: banks)
            erase: forget percent for each position (size: dim)
            add: amount to remember in each position (size: dim)
        '''
        attention = f.normalize(weights, p=1, dim=0)
        forget = einsum('i,j->ij', (attention, erase.clamp(0,1)))
        decay = torch.ones_like(forget) - forget
        
        remember = einsum('i,j->ij', (attention, add))
        
        self.memory = self.memory * decay + remember
        
    
    def get_content_similarity(self, query, key_strength = 1):
        ''' Provides similarity of query with each memory cell
        
        Args:
            query: reference memory to search for (size: dim)
            key_strength: scalar to amplify/attenuate attention
        '''
        similarity = f.cosine_similarity(self.memory, query, dim=1)
        content_attention = f.softmax(key_strength * similarity, dim=0)
        return content_attention

In [110]:
class MemoryNN(nn.Module):
    def __init__(self, hidden_size, memory_banks, memory_dim):
        ''' Initialize the Memory NN
        
        Args:
            hidden_size: input size of the LSTM hidden layer
            memory_banks: count of memory banks to include
            memory_dim: dimension of each memory bank
        '''
        super(MemoryNN, self).__init__()
        self.hidden_size = hidden_size
        self.banks = memory_banks
        self.dim = memory_dim
        
        self.memory = Memory(memory_banks, memory_dim)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=0)
        
        # For key_strength(1), sharpening(1), shift(3), key, erase, add
        self.hidden_to_dim = nn.Linear(hidden_size, 1+1+3+3*memory_dim)
        self.hidden_to_gate = nn.Linear(hidden_size, memory_banks)
        
    
    def split_outputs(self, dim_output, banks_output):
        ''' Splits network output into intuitive groupings
        
        Args:
            dim_output: output in the memory_dim space
            banks_output: output in the memory_banks space
            
        Returns:
            dictionary with each conceptual group referenceable by key
        '''
        o = {}
        o['key_strength'] = dim_output[0]
        o['sharpening'] = dim_output[1]
        o['shift'] = dim_output[2:5]
        o['key'], o['erase'], o['add'] = self._split(dim_output[5:], self.dim)
        o['gate'] = banks_output
        
        return o
        
        
    def calculate_memory_weight(self, w_old, k, β, g, s, γ):
        ''' Calculates the read/write weight for the memory
        
        Args:
            w_old: output weight from last processes token
            k: content key to use for similarity lookup in memory
            β: coef to attenuate/amplify content key attention
            g: interpolation gate to raise/lower mix of w_c/w_old
            s: shift weights to convolve with w_g
            γ: sharpening exponent for final weights
        '''
        w_c = self.memory.get_content_similarity(k, β)
        w_g = self._interpolate(w_c, w_old, g)
        w_tilde = self._convolve(w_g, s)
        weight = self._sharpen(w_tilde, γ)
        
        return weight
    
        
    def forward(self, hidden, last_weight):
        ''' Runs the Memory network using the hidden layer output
        
        Args:
            hidden: hidden layer output from LSTM to use as input
        '''
        dim_output = self.hidden_to_dim(hidden).squeeze()
        gate_output = self.hidden_to_gate(hidden).squeeze()
        
        split_output = self.split_outputs(dim_output, gate_output)
        split_output['shift'] = self.softmax(split_output['shift'])
        split_output['gate'] = self.sigmoid(split_output['gate'])
        
        weight = self.calculate_memory_weight(
            w_old = last_weight,
            k = split_output['key'],
            β = split_output['key_strength'],
            g = split_output['gate'],
            s = split_output['shift'],
            γ = split_output['sharpening']
        )
        
        read_weight = self.memory.read(weight)
        
        self.memory.write(
            weight,
            split_output['erase'],
            split_output['add']
        )
        
        return read_weight
    
    
    def reset_memory(self):
        ''' Resets the memory network for a new sequence
        
        Returns:
            pair of (initial_memory, initial_weights)
        '''
        self.memory = Memory(self.banks, self.dim)
        memory_read = torch.zeros(self.dim)
        weight = torch.zeros(self.banks)
        
        return (memory_read, weight)
    
    def _split(self, tensor, size):
        ''' Return a tensor split into chunks of length `size` '''
        count = tensor.shape[0] // size
        split = [tensor[i*size : (i+1)*size] for i in range(count)]
        return split
        
    def _interpolate(self, w1, w2, w1_mix):
        ''' Interpolate between two weights '''
        return w1_mix*w1 + (1-w1_mix)*w2
    
    def _sharpen(self, w, exponent):
        ''' Sharpen weight distribution '''
        w_sharpened = w ** exponent
        return w_sharpened / w_sharpened.sum()
    
    def _convolve(self, x, s):
        ''' Implement circular convolution between w and s '''
        # Create circular padding and reshape for conv2d
        x2 = torch.cat((x[-1:], x, x[:1])).view(1,1,1,-1)
        s2 = s.view(1,1,1,-1)

        # Run circular convolution and undo reshaping
        out = f.conv2d(x2, s2, padding=(0,1)).view(-1)

        # Return without padding
        return out[1:-1]

In [111]:
class NTM_LSTM(nn.Module):
    '''  '''
    
    def __init__(self, input_size, hidden_size, output_size, memory_banks, memory_dim):
        ''' Init the NTM-LSTM '''
        super(NTM_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.memory_banks = memory_banks
        self.memory_dim = memory_dim
        
        # Core LSTM and Memory
        self.lstm = nn.LSTM(input_size + memory_dim, hidden_size)
        self.mem_nn = MemoryNN(hidden_size, memory_banks, memory_dim)
        
        # Output Layer
        self.hidden_to_output = nn.Linear(hidden_size, output_size)
        self.output_softmax = nn.LogSoftmax(dim=2)
        
    
    def forward(self, input, state):
        ''' Run a forward path for a single number of sequence of numbers
        
        Args:
            state: contains [hidden, cell, memory, last_weight]
        
        Returns:
            output and hidden layer after last sequence input
        '''
        
        hidden, cell, memory, weight = state
        
        for i in range(input.shape[0]):
            ntm_input = torch.cat((input[i].unsqueeze(0), memory.view(1,1,-1)), dim=2)
        
            _, (hidden, cell) = self.lstm.forward(ntm_input, (hidden, cell))
            memory, weight = self.mem_nn.forward(hidden, weight)
        
        output = self.hidden_to_output(hidden)
        output = self.softmax(output)
        
        return output, (hidden, cell, memory, weight)
    
    
    def init_hidden(self):
        ''' Returns new hidden layers for the start of a new sequence '''
        memory, weight = self.mem_nn.reset_memory()
        
        model_device = next(self.parameters()).device

        return (
            torch.randn(1, 1, self.hidden_size).to(model_device),
            torch.randn(1, 1, self.hidden_size).to(model_device),
            memory.to(model_device),
            weight.to(model_device)
        )

## Problem Setup

In [112]:
# Number Generator + Goal Setup
max_number = 9
goal_func = lambda stream: stream[0]
goal_dim = max_number+2

number_tool = Numbers(max_number,
                      reset_value_func = lambda x: randint(0,max_number),
                      goal_func = goal_func)

In [98]:
# Training Size
train_size = 100e3
criterion = nn.NLLLoss()

In [99]:
setup_kwargs = {
    'train_size': train_size,
    'number_tool': number_tool,
    'criterion': criterion,
    'stream_size': 200,
    'model': None,
    'optim': None
}

In [100]:
device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

## Basic Vanilla RNN Model

In [ ]:
rnn = BasicRNN(number_tool.get_dim(), 5, goal_dim)
rnn_optim = torch.optim.SGD(rnn.parameters(), lr = 0.001)

rnn_kwargs = setup_kwargs.copy()
rnn_kwargs['model'] = rnn
rnn_kwargs['optim'] = rnn_optim

In [ ]:
train_model(**rnn_kwargs)

## Basic LSTM Model

In [15]:
lstm = BasicLSTM(number_tool.get_dim(), 5, goal_dim)
lstm.to(device)

lstm_optim = torch.optim.SGD(lstm.parameters(), lr = 0.001, momentum = 0.9)

lstm_kwargs = setup_kwargs.copy()
lstm_kwargs['model'] = lstm
lstm_kwargs['optim'] = lstm_optim

In [ ]:
train_model(**lstm_kwargs)

## Neural Turing Machine Model

In [113]:
memory_banks = 10
memory_dim = 20

ntm = NTM_LSTM(number_tool.get_dim(), 5, goal_dim, memory_banks, memory_dim)
ntm_optim = torch.optim.SGD(ntm.parameters(), lr = 0.001)

ntm_kwargs = setup_kwargs.copy()
ntm_kwargs['model'] = ntm
ntm_kwargs['optim'] = ntm_optim

In [115]:
%debug
train_model(**ntm_kwargs)

> /opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py(2219)cosine_similarity()
   2217     w12 = torch.sum(x1 * x2, dim)
   2218     w1 = torch.norm(x1, 2, dim)
-> 2219     w2 = torch.norm(x2, 2, dim)
   2220     return w12 / (w1 * w2).clamp(min=eps)
   2221 



ipdb>  up


> <ipython-input-109-4106a6d0b543>(53)get_content_similarity()
     51             key_strength: scalar to amplify/attenuate attention
     52         '''
---> 53         similarity = f.cosine_similarity(self.memory, query, dim=1)
     54         content_attention = f.softmax(key_strength * similarity, dim=0)
     55         return content_attention



ipdb>  self.memory.shape


torch.Size([10, 20])


ipdb>  query.shape


torch.Size([20])


ipdb>  q


RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [90]:
'''
x_pad = torch.cat((x[-1:], x, x[:1]))

s = s.view(1,1,1,3)
x_pad = x_pad.view(1,1,1,8)

out = f.conv2d(x_pad, s, padding=(0,1))

print(s)
print(x)
#print(x_pad)
#print(out)
print(out.view(-1)[1:-1])
'''

'\nx_pad = torch.cat((x[-1:], x, x[:1]))\n\ns = s.view(1,1,1,3)\nx_pad = x_pad.view(1,1,1,8)\n\nout = f.conv2d(x_pad, s, padding=(0,1))\n\nprint(s)\nprint(x)\n#print(x_pad)\n#print(out)\nprint(out.view(-1)[1:-1])\n'

In [91]:
s = torch.tensor([1.,0.,0.])
x = torch.tensor([0.40, .05, .50, 0, 0, .05])

print(x)
_convolve(x,s)

tensor([0.4000, 0.0500, 0.5000, 0.0000, 0.0000, 0.0500])


tensor([0.0500, 0.4000, 0.0500, 0.5000, 0.0000, 0.0000])

### ----------

-----
## Functional Testing
-----

In [168]:
# READ_MEMORY TESTS
mem = Memory(10,4)

flat = torch.ones(10)
just_4 = torch.zeros(10)
just_4[4] = 1

act = mem.read(just_4)
exp = mem.memory[4]
print(f'[RM TEST1] act:{act} exp:{exp}')

act = mem.read(flat)
exp = mem.memory.mean(dim=0)
print(f'[RM TEST2] act:{act} exp:{exp}')

# WRITE_MEMORY TESTS
mem_dim = 3

att_flat = torch.ones(10)
att_just_4 = torch.zeros(10)
att_just_4[4] = 1

forget_ones = torch.ones(mem_dim)
forget_half = torch.ones(mem_dim) / 2
forget_zeros = torch.zeros(mem_dim)

add_zero = torch.zeros(mem_dim)
add_ones = torch.ones(mem_dim)

mem = Memory(10, mem_dim)
exp = mem.memory[4]
act = mem.write(att_just_4, forget_zeros, add_zero)[4]
print(f'[WM TEST1] act:{act} exp:{exp}')

mem = Memory(10, mem_dim)
exp = mem.memory[4] + 1
act = mem.write(att_just_4, forget_zeros, add_ones)[4]
print(f'[WM TEST2] act:{act} exp:{exp}')

mem = Memory(10, mem_dim)
exp = torch.zeros_like(mem.memory[4])
act = mem.write(att_just_4, forget_ones, add_zero)[4]
print(f'[WM TEST3] act:{act} exp:{exp}')

mem = Memory(10, mem_dim)
exp = mem.memory * (1 - 0.5 / 10)
act = mem.write(att_flat, forget_half, add_zero)
print(f'[WM TEST4] act:{act.view(-1)} exp:{exp.view(-1)}')

mem = Memory(10, mem_dim)
exp = mem.memory * (1 - 0.5 / 10) + 0.1
act = mem.write(att_flat, forget_half, add_ones)
print(f'[WM TEST5] act:{act.view(-1)} exp:{exp.view(-1)}')

[RM TEST1] act:tensor([ 0.5261, -1.5163, -0.6044,  0.5245]) exp:tensor([ 0.5261, -1.5163, -0.6044,  0.5245])
[RM TEST2] act:tensor([ 0.1845, -0.4467,  0.1367, -0.5949]) exp:tensor([ 0.1845, -0.4467,  0.1367, -0.5949])
[WM TEST1] act:tensor([-0.9003,  0.9227, -0.7564]) exp:tensor([-0.9003,  0.9227, -0.7564])
[WM TEST2] act:tensor([1.1107, 3.1953, 1.5835]) exp:tensor([1.1107, 3.1953, 1.5835])
[WM TEST3] act:tensor([0., 0., 0.]) exp:tensor([0., 0., 0.])
[WM TEST4] act:tensor([ 1.8990, -0.5792,  0.9731,  0.5017, -0.8982,  0.1428,  1.1492, -0.0978,
         0.2197,  1.9246,  1.2118, -1.0656,  1.6490, -0.5932, -0.5040,  1.5088,
         0.0564,  2.4903, -0.4258, -0.0668, -1.1677, -0.6731, -1.2875,  1.8346,
        -1.0690,  0.2515,  1.2229, -0.3647,  1.2093, -1.9404]) exp:tensor([ 1.8990, -0.5792,  0.9731,  0.5017, -0.8982,  0.1428,  1.1492, -0.0978,
         0.2197,  1.9246,  1.2118, -1.0656,  1.6490, -0.5932, -0.5040,  1.5088,
         0.0564,  2.4903, -0.4258, -0.0668, -1.1677, -0.6731, -